In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
nltk.download('all-nltk')

In [ ]:
#cleaning Process Libraries
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd

In [ ]:
messages = pd.read_csv('/kaggle/input/smsspamcollection/SMSSpamCollection', sep = '\t', names = ['label', 'message'])

In [ ]:
messages

In [ ]:
stemmer = PorterStemmer()
cleanedSentence = []

In [ ]:
# Clean the Data
for i in range(len(messages)):
    sent = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    sent = sent.lower()
    sent = sent.split()
    sent = [stemmer.stem(word) for word in sent if word not in set(stopwords.words('english'))]
    sent = ' '.join(sent)
    cleanedSentence.append(sent)

In [ ]:
cleanedSentence

In [ ]:
# Implement Bag of words Model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(cleanedSentence).toarray()

In [ ]:
X

In [ ]:
y = pd.get_dummies(messages['label'])

In [ ]:
y

In [ ]:
y = y.iloc[:,1].values

In [ ]:
y

In [ ]:
#removing class Imbalance
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy='auto')
X,y = sm.fit_resample(X,y)

In [ ]:
pip install --user pycaret

In [ ]:
X = pd.DataFrame(X)
y = pd.DataFrame(y)

In [ ]:
y

In [ ]:
y = y.rename(columns={0: 'Spam'})

In [ ]:
y

In [ ]:
data = pd.concat([X,y], axis = 1)

In [ ]:
data

In [ ]:
from pycaret.classification import *

In [ ]:
clf = setup(data=data, target = 'Spam', session_id = 123, imputation_type='iterative')

In [ ]:
compare_models(sort = 'AUC')

In [ ]:
rf = create_model('rf')

In [ ]:
tuned_rf = tune_model(rf, optimize = 'AUC')

In [ ]:
final_rf = finalize_model(tuned_rf)
predict_model(final_rf)

In [ ]:
prediction = predict_model(final_rf, data=data)
prediction.head()